<a href="https://colab.research.google.com/github/Kannant08/Machine-Leaning/blob/main/RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
|from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import fitz  # PyMuPDF
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import TextLoader
from langchain.docstore.document import Document
from langchain_groq import ChatGroq

In [ ]:
pdf_path = "/kebo101.pdf"

In [ ]:
groq_api_key = "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"

In [ ]:
#llm = ChatGroq(groq_api_key = groq_api_key, model_name = "Gemma-7b-it")

In [ ]:
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text

In [ ]:
def split_text(text):
    text_splitter = CharacterTextSplitter(
        separator="\n",
        chunk_size=1000,
        chunk_overlap=100,
        length_function=len,
    )
    return text_splitter.split_text(text)

In [ ]:
def create_documents(chunks):
    return [Document(page_content=chunk) for chunk in chunks]

In [ ]:
def create_retriever(documents):
    embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
    vectorstore = FAISS.from_documents(documents, embeddings)
    return vectorstore.as_retriever()

In [ ]:
def setup_qa_chain(retriever):
    #llm = OpenAI(temperature=0)
    llm = ChatGroq(groq_api_key = groq_api_key, model_name = "Gemma-7b-it")
    prompt_template = """Given the context provided and a question, generate an answer strictly based on the given context.
Use as much of the text from the "response" section in the source document as possible, with minimal alterations.
If the answer cannot be found in the context, respond with "I don't know." Do not fabricate an answer.

    CONTEXT: {context}

    QUESTION: {question}"""

    PROMPT = PromptTemplate(
        template=prompt_template, input_variables=["context", "question"]
    )
    chain_type_kwargs = {"prompt": PROMPT}

    chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=retriever,
        input_key="query",
        return_source_documents=True,
        chain_type_kwargs=chain_type_kwargs,
    )
    return chain

In [ ]:
def main(pdf_path, question):

    text = extract_text_from_pdf(pdf_path)

    chunks = split_text(text)

    documents = create_documents(chunks)

    retriever = create_retriever(documents)

    qa_chain = setup_qa_chain(retriever)

    result = qa_chain({"query": question})
    print("Answer:", result['result'])
    print("\nSource Documents:")
    for doc in result['source_documents']:
        print(doc.page_content[:500])

In [35]:
if __name__ == "__main__":
    pdf_path = pdf_path  # Replace with your PDF file path
    question = "what is taxanomy"
    main(pdf_path, question)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Answer: Taxonomy is the process of classifying living organisms based on their characteristics, structure, and evolutionary relationships. It involves characterization, identification, classification, and nomenclature of organisms, with the ultimate goal of understanding their diversity and relationships.

Source Documents:
talking about. Similarly, suppose we were to say ‘mammals’, you would,
of course, think of animals with external ears and body hair. Likewise, in
plants, if we try to talk of ‘Wheat’, the picture in each of our minds will be
of wheat plants, not of rice or any other plant. Hence, all these - ‘Dogs’,
‘Cats’, ‘Mammals’, ‘Wheat’, ‘Rice’, ‘Plants’, ‘Animals’, etc., are convenient
categories we use to study organisms. The scientific term for these
categories is taxa. Here you must recognise that taxa
into different taxa. This process of classification is taxonomy. External
and internal structure, along with the structure of cell, development
process and ecological inform